In [1]:
# 1️⃣ 라이브러리 및 경로 설정
import os
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import linear
from math import radians, sin, cos, sqrt, atan2

# 📁 파일 경로 설정
shp_path = "BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"
base_path = "../../../data/Combine/"
child_path = os.path.join(base_path, "Child")
cctv_path = os.path.join(base_path, "CCTV")

In [2]:
# 2️⃣ SHP 파일 로딩 및 어린이 인구 비율 매핑
# SHP 전체 로드
gdf_all = gpd.read_file(shp_path)

# 행정동명 정리 ("제" 제거)
def remove_je(name):
    return name.replace("제", "") if "제" in name else name

name_mapping_raw = {
    "화양동": 0.01, "군자동": 0.04, "중곡제1동": 0.03,
    "중곡제2동": 0.04, "중곡제3동": 0.04, "중곡제4동": 0.04,
    "능동": 0.04, "광장동": 0.11, "자양제1동": 0.04,
    "자양제2동": 0.05, "자양제3동": 0.08, "자양제4동": 0.04,
    "구의제1동": 0.04, "구의제2동": 0.06, "구의제3동": 0.06,
}

# "제" 제거 후 실제 SHP에 있는 동과 일치시키기
name_mapping_full = {remove_je(k): v for k, v in name_mapping_raw.items()}
existing_dongs = set(gdf_all["ADM_NM"]) & set(name_mapping_full.keys())
gdf = gdf_all[gdf_all["ADM_NM"].isin(existing_dongs)].copy()
name_mapping = {dong: name_mapping_full[dong] for dong in existing_dongs}
gdf["어린이비율"] = gdf["ADM_NM"].map(name_mapping)

print(f"\U0001F4CC 실제 SHP에 존재하는 광진구 행정동 수: {len(gdf)}")
print(f"⚠️ 매핑 안 된 행정동 수: {gdf['어린이비율'].isnull().sum()}")

📌 실제 SHP에 존재하는 광진구 행정동 수: 16
⚠️ 매핑 안 된 행정동 수: 0


In [3]:
# 3️⃣ 보호구역 및 CCTV 데이터 불러오기
child_df = pd.read_csv(os.path.join(child_path, "gwangjin_child_zone_with_coords_filtered.csv"))
cctv_df = pd.read_csv(os.path.join(cctv_path, "proceeded_cctv.csv"))

In [4]:
# 4️⃣ 지도 초기화
m = folium.Map(location=[37.5386, 127.0822], zoom_start=13)

# 색상 범례
colormap = linear.YlOrRd_09.scale(gdf["어린이비율"].min(), gdf["어린이비율"].max())
colormap.caption = "행정동별 만3~12세 어린이 비율"

# 행정동 색상 표시
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["어린이비율"]),
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["ADM_NM", "어린이비율"],
        aliases=["행정동", "어린이 비율"],
        localize=True
    )
).add_to(m)
colormap.add_to(m)


In [5]:
# 5️⃣ 거리 계산 함수 (하버사인 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = radians(lat1), radians(lat2)
    d_phi = radians(lat2 - lat1)
    d_lambda = radians(lon2 - lon1)
    a = sin(d_phi/2)**2 + cos(phi1)*cos(phi2)*sin(d_lambda/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

In [6]:
# 6️⃣ 보호구역 300m 원 + CCTV 개수 표시
for _, row in child_df.iterrows():
    lat_c, lon_c = row["위도"], row["경도"]

    # CCTV 개수 세기
    nearby_count = cctv_df.apply(
        lambda c: haversine(lat_c, lon_c, c["위도"], c["경도"]) <= 300,
        axis=1
    ).sum()

    # 원 그리기
    folium.Circle(
        location=[lat_c, lon_c],
        radius=300,
        color="blue",
        fill=True,
        fill_opacity=0.2,
        popup=f"{row.get('시설명', '보호구역')}<br>CCTV 개수: {nearby_count}"
    ).add_to(m)

In [7]:
# 7️⃣ CCTV 마커 표시
for _, row in cctv_df.iterrows():
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.5
    ).add_to(m)

# 8️⃣ 저장
output_path = os.path.abspath("아동위험지도.html")
m.save(output_path)
print("✅ 저장 완료:", output_path)


✅ 저장 완료: /home/wsl/code/gongmo/src/notebooks/stundrg/merge_data/아동위험지도.html
